# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json
from pprint import pprint
%matplotlib inline
from scipy.stats import linregress
from matplotlib import pyplot as plt
from api_keys import g_key 

# Import API key
import api_keys as api

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_path = 'city_weather.csv'
vacation_df = pd.read_csv(vacation_path)
vacation_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,48.99,88,0,8.12,AU,1618696356
1,1,New Norfolk,-42.7826,147.0587,55.99,79,100,5.01,AU,1618696286
2,2,Grindavik,63.8424,-22.4338,37.40,70,90,13.80,IS,1618696357
3,3,Vaini,-21.2000,-175.2000,77.00,73,20,10.36,TO,1618696288
4,4,Halifax,44.6453,-63.5724,42.01,100,100,16.11,CA,1618696202
...,...,...,...,...,...,...,...,...,...,...
546,546,Khunzakh,42.5394,46.7050,44.20,84,96,0.27,RU,1618696523
547,547,Ola,59.5833,151.2833,24.89,87,88,6.31,RU,1618696524
548,548,Teknāf,20.8624,92.3058,74.59,83,1,3.11,BD,1618696524
549,549,Warri,5.5167,5.7500,78.37,90,86,3.94,NG,1618696524


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = vacation_df[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
# HINT: be sure to handle NaN values
#vacation_df = vacaton_df.dropna()
humidity_perc = vacation_df.Humidity.astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_perc, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
vacation_df["Cloudiness"].value_counts()

100    88
0      69
90     41
75     39
1      36
       ..
34      1
32      1
31      1
28      1
53      1
Name: Cloudiness, Length: 89, dtype: int64

In [5]:
max_temp = 85
min_temp = 75
ideal_vacay_df = vacation_df.loc[(vacation_df["Max Temp"] <= max_temp) & (vacation_df["Max Temp"] >= min_temp) &(vacation_df.Cloudiness <=1) & (vacation_df["Wind Speed"]<=10)]
ideal_vacay_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,Riyadh,24.6877,46.7219,77.00,23,0,5.75,SA,1618696364
87,87,Carnarvon,-24.8667,113.6333,75.63,62,1,5.30,AU,1618696127
89,89,Mathbaria,22.2876,89.9593,76.53,88,0,1.70,BD,1618696382
118,118,Portland,45.5234,-122.6762,82.99,22,1,5.75,US,1618696232
132,132,Marsá Maţrūḩ,31.3525,27.2453,77.00,27,0,9.22,EG,1618696396
196,196,Mossamedes,-15.1961,12.1522,81.72,68,0,4.43,AO,1618696415
299,299,Karratha,-20.7377,116.8463,79.79,47,0,7.65,AU,1618696448
339,339,Noumea,-22.2763,166.4572,78.01,78,0,1.14,NC,1618696460
360,360,Burhānpur,21.3000,76.2333,84.96,24,1,1.99,IN,1618696466
368,368,Atar,20.5169,-13.0499,79.27,18,0,9.15,MR,1618696468


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_vacay_df.dropna(inplace=True)
ideal_vacay_df

<ipython-input-6-a2923cc032d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_vacay_df.dropna(inplace=True)


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,Riyadh,24.6877,46.7219,77.00,23,0,5.75,SA,1618696364
87,87,Carnarvon,-24.8667,113.6333,75.63,62,1,5.30,AU,1618696127
89,89,Mathbaria,22.2876,89.9593,76.53,88,0,1.70,BD,1618696382
118,118,Portland,45.5234,-122.6762,82.99,22,1,5.75,US,1618696232
132,132,Marsá Maţrūḩ,31.3525,27.2453,77.00,27,0,9.22,EG,1618696396
196,196,Mossamedes,-15.1961,12.1522,81.72,68,0,4.43,AO,1618696415
299,299,Karratha,-20.7377,116.8463,79.79,47,0,7.65,AU,1618696448
339,339,Noumea,-22.2763,166.4572,78.01,78,0,1.14,NC,1618696460
360,360,Burhānpur,21.3000,76.2333,84.96,24,1,1.99,IN,1618696466
368,368,Atar,20.5169,-13.0499,79.27,18,0,9.15,MR,1618696468


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#hotel_name = []
#ideal_vacay_df.insert(9, "Hotel Name", True)
hotel_df = ideal_vacay_df[["City", "Country", "Lat", "Lng", "Max Temp", "Humidity"]]
#hotel_df["Hotel Name"]=""
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity
26,Riyadh,SA,24.6877,46.7219,77.00,23
87,Carnarvon,AU,-24.8667,113.6333,75.63,62
89,Mathbaria,BD,22.2876,89.9593,76.53,88
118,Portland,US,45.5234,-122.6762,82.99,22
132,Marsá Maţrūḩ,EG,31.3525,27.2453,77.00,27
196,Mossamedes,AO,-15.1961,12.1522,81.72,68
299,Karratha,AU,-20.7377,116.8463,79.79,47
339,Noumea,NC,-22.2763,166.4572,78.01,78
360,Burhānpur,IN,21.3000,76.2333,84.96,24
368,Atar,MR,20.5169,-13.0499,79.27,18


In [8]:
#cities = ideal_vacay_df.City
lat = hotel_df.Lat
lat
lng = hotel_df.Lng
#lng

In [9]:
# params dictionary to update each iteration
#hotel_name = []

params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": api.g_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
       # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_search = requests.get(base_url, params=params)
    
    # convert to json
    hotel_search = hotel_search.json()

    #print(json.dumps(hotel_search, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        #create and store hotel name and hotel rating
        hotel_df.loc[index, "Hotel Name"] = hotel_search["results"][0]["name"]
        hotel_df.loc[index, "Hotel Rating"] = hotel_search["results"][0]["rating"]
       
        #hotel_name.append(hotel_search["results"][0]["name"])
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        #hotel_name.append(np.nan(inplace=True))

hotel_df

C:\Users\bethg\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\bethg\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,City,Country,Lat,Lng,Max Temp,Humidity,Hotel Name,Hotel Rating
26,Riyadh,SA,24.6877,46.7219,77.00,23,Muhaidib Hotel,3.8
87,Carnarvon,AU,-24.8667,113.6333,75.63,62,Carnarvon Hotel,4.3
89,Mathbaria,BD,22.2876,89.9593,76.53,88,Hotel Grand Park Barisal,4.3
118,Portland,US,45.5234,-122.6762,82.99,22,The Heathman Lodge,4.5
132,Marsá Maţrūḩ,EG,31.3525,27.2453,77.00,27,Almaza Bay,4.6
196,Mossamedes,AO,-15.1961,12.1522,81.72,68,iu Hotel Namibe,3.6
299,Karratha,AU,-20.7377,116.8463,79.79,47,Karratha International Hotel,4.0
339,Noumea,NC,-22.2763,166.4572,78.01,78,DoubleTree Noumea Ilot Maitre resort,4.3
360,Burhānpur,IN,21.3000,76.2333,84.96,24,Hotel High Rise Regency,4.9
368,Atar,MR,20.5169,-13.0499,79.27,18,Auberge mer et desert,4.1


In [10]:
hotel_df.to_csv("hotel.csv")

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [12]:
#hotel_info
#locations

In [13]:

gmaps.configure(api_key=g_key)
locations = hotel_df[["Lat", "Lng"]].astype(float)

humidity_perc = hotel_df.Humidity.astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_perc, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

#fig.add_layer(heat_layer)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))